## PingOne Overview

PingOne is a cloud-based identity and access management service that provides secure identity solutions for enterprises, enabling seamless authentication and authorization across applications and services.

Key Features:
* **Single Sign-On (SSO)** - Users authenticate once to access multiple applications
* **Multi-Factor Authentication (MFA)** - Enhanced security through additional verification methods
* **Adaptive Authentication** - Risk-based authentication policies based on user behavior and context
* **Universal Directory** - Centralized user management and profile synchronization
* **API Access Management** - OAuth 2.0 and OpenID Connect support for API security

## Learning Objective
PingOne can be used as an identity provider on AgentCore Identity and used to authenticate users and have them authorize the agent to access protected resources on their behalf. In this notebook we will explore the use of PingOne for inbound authentication - Authenticate users before they can invoke an agent.

## Tutorial Architecture

<figure style="width: 70%;">
    <img src="/images/inbound_auth_diagram.png" style="width: 70%; height: auto;" alt="Detailed AWS Cloud Architecture">
</figure>

## Tutorial Details

| Information         | Details                                                                          |
|:--------------------|:---------------------------------------------------------------------------------|
| Tutorial type       | Conversational                                                                   |
| Agent type          | Single                                                                           |
| Agentic Framework   | Strands Agents                                                                   |
| LLM model           | Anthropic Claude Sonnet 3.5                                                      |
| Tutorial components | Hosting agent on AgentCore Runtime. Using Strands Agent and Amazon Bedrock Model |
| Tutorial vertical   | Cross-vertical                                                                   |
| Example complexity  | Easy                                                                             |
| Inbound Auth        | PingOne                                                                          |
| SDK used            | Amazon BedrockAgentCore Python SDK and boto3                                     |

## Prerequisites

- PingOne environment
- PingOne IAM rights to create new IAM resources, applications, and users
- Python 3.10+
- AWS Credentials
- AWS IAM rights to create a new AgentCore Agent
- Amazon Bedrock AgentCore SDK
- Strands Agents
- AWS region that supports Bedrock AgentCore. Refer https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/agentcore-regions.html

## Learning Objective 1 - Setup PingOne for use with AgentCore

### 1.1: Create a New Resource

1. Login to your PingOne environment
2. Select **Applications**, then **Resources**, and add a resource
3. Configure the resource:
    - For **Resource Name** and **Audience**, enter **agentcore_runtime**
    - For **Scopes**, add **agent:invoke**

<figure>
    <img src="/images/inbound_resource_config.png">
</figure>

### 1.2: Create a New Application

1. Select **Applications**, then **Applications**, and add an application
2. Select **OIDC Web App** as the **Application Type**
3. Configure the application:
    - For **Application Name**, enter **Amazon Bedrock Chat App**
    - For **Token Auth Method**, select **Client Secret Basic**
    - For **Grant Type**, select **Authorization Code** and **Device Authorization**
        - **Note:** While Authorization Code is the standard for web apps, we use the Device Authorization grant in this notebook to allow for seamless user authentication within the Python environment without requiring a local redirect listener.
    - For **Redirect URIs**, use "https://bedrock-agentcore.your-aws-region.amazonaws.com/identities/oauth2/callback"
    - For **Resources**, add the **agent:invoke** scope on the **agentcore_runtime** resource configured in step 1.1

<figure>
    <img src="/images/inbound_application_config.png">
</figure>
    

### 1.3: Create a Test User

1. Select **Directory**, then **Users** and add a user
2. Fill in the details of the test user that will be used for this demo

<figure>
    <img src="/images/inbound_user_config.png">
</figure>

## Learning Objective 2 - Setup a Simple Agent with PingOne for Inbound Authentication

### 2.1: Install Dependencies

In [ ]:
# Install required packages from requirements.txt
%pip install --force-reinstall -U -r requirements.txt --quiet

### 2.2: Configure Environment Variables

In [ ]:
import os

# PingOne Configuration - Replace with your values
os.environ['PINGONE_CLIENT_ID'] = 'YOUR_CLIENT_ID_VALUE'
os.environ['PINGONE_CLIENT_SECRET'] = 'YOUR_CLIENT_SECRET_VALUE'
os.environ['PINGONE_AUDIENCE'] = 'agentcore_runtime'
os.environ['PINGONE_SCOPE'] = 'openid agent:invoke'
os.environ['PINGONE_TOKEN_URL'] = 'https://auth.pingone.your-region/your-environment/as/token'
os.environ['PINGONE_DEVICE_AUTH_URL'] = 'https://auth.pingone.your-region/your-environment/as/device_authorization'
os.environ['PINGONE_DISCOVERY_URL'] = 'https://auth.pingone.your-region/your-environment/as/.well-known/openid-configuration'

### 2.3: Discover AWS Environment Identity

In [ ]:
import boto3
from boto3.session import Session

boto_session = Session()
sts = boto3.client('sts')
region = boto_session.region_name
account_id = sts.get_caller_identity().get("Account")
print(f"📍 Deploying to Account: {account_id} in Region: {region}")

### 2.4: Agent Code
Keeping the agent simple since the key learning objective for this notebook is to learn inbound authentication using PingOne

In [ ]:
%%writefile simple_agent.py
import argparse, json
from strands import Agent, tool
from bedrock_agentcore.runtime import BedrockAgentCoreApp

app = BedrockAgentCoreApp()
agent = Agent()

@app.entrypoint
def invoke(payload):
    # Capture what the user actually said
    user_message = payload.get("prompt", "Hello!")
    session_id = payload.get("session_id", "no-session")

    # Create the prompt for the agent
    agent_prompt = f"User (Session {session_id}) asks: {user_message}"

    # Send that prompt to the agent
    agent_response = agent(agent_prompt)

    # Return only the agents answer
    return {"result": agent_response.message}

if __name__ == "__main__":
    app.run()

### 2.5: Secure the Runtime Environment with PingOne Authentication

This configuration creates a JWT Authorizer that delegates identity verification to PingOne, ensuring only requests with the correct Audience and Scopes can reach your agent.

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime

# Init runtime object that handles deployment and infra of the bedrock agent
agentcore_runtime = Runtime()

# Configure the agent environment and deployment settings
try:
    response = agentcore_runtime.configure(
        entrypoint="simple_agent.py",
        auto_create_execution_role=True,
        auto_create_ecr=True,
        requirements_file="requirements.txt",
        region=region,
        agent_name="pingone_inbound_auth_agent",
        # Secure the agent's API with a custom JWT Authorizer using PingOne as the IdP
        authorizer_configuration={
            "customJWTAuthorizer": {
                "discoveryUrl": os.getenv('PINGONE_DISCOVERY_URL'),
                "allowedClients": [os.getenv('PINGONE_CLIENT_ID')],
                "allowedAudience": [os.getenv('PINGONE_AUDIENCE')],
                "allowedScopes": (os.getenv('PINGONE_SCOPE') or "openid agent:invoke").split()
            }
        }
    )
    print("✅ OAuth configuration successful")
except Exception as e:
    print(f"❌ OAuth configuration failed: {e}")

### 2.6: Deploy the Authenticated Agent

This step executes the final deployment by linking the agent logic (from 2.4) with the security rules (from 2.5). The toolkit packages your code into a container, pushes it to ECR, and provisions the serverless AgentCore Runtime endpoint.

In [ ]:
# Package code and push to AWS ECR, triggering the deployment and infrastructure setup
launch_result = agentcore_runtime.launch(auto_update_on_conflict=True)

### 2.7: Check the AgentCore Runtime Status

Monitor the deployment status until the agent is ready.

In [ ]:
import time

status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
while status not in ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)

status

### 2.8: Save Agent ARN for Testing

Extract and save the Agent ARN for testing purposes.

In [ ]:
# Extract ARN from launch_result
if hasattr(launch_result, 'agent_arn') and launch_result.agent_arn:
    agent_arn = launch_result.agent_arn
    os.environ['AGENT_ARN'] = agent_arn
    print(f"📝 Agent ARN: {agent_arn}")
    print(f"📝 Agent ID: {launch_result.agent_id}")
    print(f"📝 ECR URI: {launch_result.ecr_uri}")
else:
    print("⚠️  Could not extract Agent ARN from launch result")
    print("Launch result:", launch_result)

# Check deployment status
if status == 'READY':
    print("✅ Agent deployed successfully and ready for testing!")
elif status in ['CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']:
    print(f"❌ Agent deployment failed with status: {status}")
else:
    print(f"⚠️  Unexpected status: {status}")

## Learning Objective 3 - Prepare for Agent Authentication and Testing

### 3.1 Define the PingOne Authentication Handler

We will use the device authorization grant here for demo purposes

In [ ]:
import requests
import time
import webbrowser
from requests.auth import HTTPBasicAuth

def get_pingone_token():
    # Initialize device authorization flow
    auth_res = requests.post(
        os.environ['PINGONE_DEVICE_AUTH_URL'], 
        data={
            "client_id": os.environ['PINGONE_CLIENT_ID'],
            "scope": os.environ.get('PINGONE_SCOPE', 'openid agent:invoke')
        },
        auth=HTTPBasicAuth(
            os.environ['PINGONE_CLIENT_ID'], 
            os.environ['PINGONE_CLIENT_SECRET']
        )
    )
    auth_res.raise_for_status()
    data = auth_res.json()
    print(f"\n👉 ACTION REQUIRED")
    print(f"1. Go to: {data['verification_uri']}")
    print(f"2. Enter Code: {data['user_code']}")

    # Automatically open the browser
    webbrowser.open(data['verification_uri'])

    # 2. Polling for the Access Token
    print("\n⏳ Waiting for you to log in...")
    while True:
        interval = data.get('interval', 5)
        token_res = requests.post(
            os.environ['PINGONE_TOKEN_URL'], 
            data={
                "grant_type": "urn:ietf:params:oauth:grant-type:device_code",
                "device_code": data['device_code'],
                "client_id": os.environ['PINGONE_CLIENT_ID']
            },
            auth=HTTPBasicAuth(
                os.environ['PINGONE_CLIENT_ID'], 
                os.environ['PINGONE_CLIENT_SECRET']
            )
        )
        res_data = token_res.json()
        if "access_token" in res_data:
            print("✅ Successfully authenticated!")
            return res_data['access_token']
        error = res_data.get("error")
        if error == "authorization_pending":
            time.sleep(interval)
        elif error == "slow_down":
            time.sleep(interval + 2)
        else:
            raise Exception(f"Authentication failed: {res_data.get('error_description', error)}")

### 3.2: Define the Function to Invoke the Agent

This is what will call the agent with the bearer token

In [ ]:
import urllib.parse
import uuid

def invoke_agent(access_token, query, session_id=None):
    escaped_agent_arn = urllib.parse.quote(os.environ['AGENT_ARN'], safe='')
    url = f"https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{escaped_agent_arn}/invocations?qualifier=DEFAULT"

    # Generate session ID if not provided
    if not session_id:
        session_id = f'pingone-inbound-session-{int(time.time())}-{uuid.uuid4().hex[:8]}'

    # Invoke the agent
    print(f"🚀 Invoking agent with query: {query}")
    print(f"📋 Session ID: {session_id}")
    response = requests.post(
        url,
        headers={
            'Authorization': f'Bearer {access_token}',
            'Content-Type': 'application/json',
            'X-Amzn-Bedrock-AgentCore-Runtime-Session-Id': session_id
        },
        json={
            'prompt': query,
            'session_id': session_id
        },
        timeout=300
    )
    response.raise_for_status()
    result = response.json()
    print("✅ Agent response received")
    return result, session_id

## Learning Objective 4 - Test the Agent with and without Authentication

### 4.1: Test Unauthenticated Request (Should Fail)

First, let's verify that our agent properly rejects unauthenticated requests:

In [ ]:
print("=" * 50)
print("TEST 0: Unauthenticated Request (Should Fail)")
print("=" * 50)

try:
    escaped_agent_arn = urllib.parse.quote(os.environ['AGENT_ARN'], safe='')
    url = f"https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{escaped_agent_arn}/invocations?qualifier=DEFAULT"
    response = requests.post(
        url,
        headers={'Content-Type': 'application/json'}, # No auth
        json={'prompt': 'Hello without authentication'}
    )
    response.raise_for_status()
    print(f"❌ Unexpected success! Status: {response.status_code}")
    print(f"Response: {response.text}")
except requests.exceptions.HTTPError as e:
    print(f"✅ Expected authentication failure: {e.response.status_code}")
    print(f"Error message: {e.response.text}")
except Exception as e:
    print(f"✅ Expected authentication error: {e}")

### 4.2: Test Authenticated Request

Now lets get a bearer token and test the agent responds to authenticated requests:

In [ ]:
import json

print("=" * 50)
print("TEST 1: Simple Query with Session ID")
print("=" * 50)

# Get OAuth token
access_token = get_pingone_token()

# Test authenticated agent invocation with session ID tracking
result1, session_id1 = invoke_agent(access_token, "Hello can you guide me about AWS security best practices for authentication?")
print(f"Session ID used: {session_id1}")
print(json.dumps(result1, indent=2))

### 4.3: Test Session Continuity by Reusing the Same Session ID

Finally lets verify our agent can hold a conversation:

In [ ]:
print("=" * 50)
print("TEST 2: Continue Conversation with Same Session")
print("=" * 50)

# Continue conversation by using the same session ID
result2, session_id2 = invoke_agent(access_token, "What was my previous question?", session_id1)
print(f"Session ID used: {session_id2}")
print(json.dumps(result2, indent=2))

## Conclusion and Cleanup

### Delete AgentCore Runtime

Clean up the resources created in this tutorial:

In [ ]:
import boto3

# Delete the AgentCore Runtime
try:
    if hasattr(launch_result, 'agent_id'):
        agentcore_control_client = boto3.client("bedrock-agentcore-control", region_name=region)
        agentcore_control_client.delete_agent_runtime(agentRuntimeId=launch_result.agent_id)
        print(f"✅ Agent {launch_result.agent_id} deleted successfully")
    else:
        print("⚠️  No agent ID found to delete")
except Exception as e:
    print(f"❌ Error deleting agent: {e}")
    print("You may need to delete the agent manually from the AWS console")

### Conclusion

This notebook demonstrated how to:
1. **Setup PingOne IDP** - Configure a PingOne environment, resource, and application
2. **Create Simple Agent** - Build a basic agent with session awareness
3. **Configure OAuth Authentication** - Set up AgentCore Runtime with PingOne JWT validation
4. **Deploy with Authentication** - Deploy agent with inbound authentication
5. **Test Authentication Flow** - Verify OAuth token flow and session management

### Key Learnings:

- **Inbound Authentication:** PingOne protects agent endpoints, ensuring only authenticated users can invoke agents
- **Session Management:** Agents can access session information for personalized responses
- **JWT Token Validation:** AgentCore automatically validates PingOne JWT tokens
- **Security:** Unauthenticated requests are automatically rejected

### Next Steps:

- Implement user-based authentication flows
- Add more sophisticated agent logic with user context
- Explore outbound authentication for accessing external APIs
- Integrate with AgentCore Gateway for additional security layers